In [8]:
from tensorflow.keras import optimizers, losses, callbacks
from tensorflow.data import TFRecordDataset
from src import models, loader


In [2]:
batch_size = 750

builder = loader.DatasetBuilder('data', shape=(256, 256))
train = TFRecordDataset('data/train.records')
train = train.map(builder.decoder)
train = train.map(builder.data_augmentation)
train = train.shuffle(16000)
train = train.batch(batch_size=batch_size)

In [3]:
test = TFRecordDataset('data/test.records')
test = test.map(builder.decoder)
test = test.map(builder.normalize)
test = test.shuffle(4000)
test = test.batch(batch_size=batch_size)

In [4]:
model = models.VggModel()
model.build((1, 256, 256, 3))
model.summary()

Model: "vgg_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  1792      
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  36928     
_________________________________________________________________
conv2d_2 (Conv2D)            multiple                  36928     
_________________________________________________________________
conv2d_3 (Conv2D)            multiple                  36928     
_________________________________________________________________
conv2d_4 (Conv2D)            multiple                  36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
conv2d_5 (Conv2D)            multiple                  36

In [7]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',  
              metrics=['accuracy'])

In [ ]:
history = model.fit(train, epochs=2, validation_data=test)

Epoch 1/2
